<a href="https://colab.research.google.com/github/M-Sravanthi/ImpactEval/blob/main/MannKendallImpactEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Step 1: Install the required package
!pip install pymannkendall
!pip install pyhomogeneity

In [ ]:
# 📌 Step 2: Import necessary libraries
import pandas as pd
import pymannkendall as mk
import pyhomogeneity as ph
# Load your CSV file (replace with correct path)
file_path = '/content/AllIndices_TimeSeries_PerSite.csv'  # Update this path
df = pd.read_csv(file_path)

# 📌 Step 4: Preview your data
print(df.head())


   Sr_no       bsi       evi      ndbi      ndvi  year  longitude   latitude
0      1 -0.269890  0.582715 -0.354647  0.730891  2018  88.425979  27.216662
1      2 -0.269890  0.582715 -0.354647  0.730891  2018  88.425979  27.216662
2      4 -0.269890  0.582715 -0.354647  0.730891  2018  88.425979  27.216662
3      3 -0.364582  0.745155 -0.416183  0.848571  2018  88.425979  27.216752
4      5 -0.262793  0.466539 -0.326654  0.835590  2018  88.429932  27.213339


In [ ]:
# 📌 Check required columns
required_columns = {'Sr_no',  'bsi', 'evi','ndbi','ndvi', 'year', 'longitude', 'latitude'}
#assert required_columns.issubset(set(df.columns)), "Missing required columns"

# 📌 Group by each site
results = []

for site, group in df.groupby('Sr_no'):
    group_sorted = group.sort_values(by='year')
    lat = group_sorted['latitude'].iloc[0]
    lon = group_sorted['longitude'].iloc[0]

    site_result = {
        'Sr_no': site,
        'latitude': lat,
        'longitude': lon
    }

    for index in ['ndvi', 'evi', 'bsi', 'ndbi']:
        series = group_sorted[index].tolist()
        val = group_sorted[index].values
        mean = val.mean()
        std = val.std()
        cv = std / mean * 100
        if len(series) >= 3:
            result = mk.original_test(series)
            pettitt = ph.pettitt_test(series)
            site_result[f'{index}_trend'] = result.trend
            site_result[f'{index}_p_value'] = result.p
            site_result[f'{index}_tau'] = result.Tau
            site_result[f'{index}_slope'] = result.slope
            site_result[f'{index}_cv'] = cv
            site_result[f'{index}_pettitt_cp']= pettitt.cp
            site_result[f'{index}_pettitt_p']= pettitt.p
            site_result[f'{index}_pettitt_u'] = pettitt.U        # Test statistic
            site_result[f'{index}_pettitt_h'] = pettitt.h        # Hypothesis test result (True/False)
        else:
            site_result[f'{index}_trend'] = 'insufficient data'
            site_result[f'{index}_p_value'] = None
            site_result[f'{index}_tau'] = None
            site_result[f'{index}_slope'] = None
            site_result[f'{index}_cv'] = None
            site_result[f'{index}_pettitt_cp']= None
            site_result[f'{index}_pettitt_p']= None
            site_result[f'{index}_pettitt_u'] = None
            site_result[f'{index}_pettitt_h'] = None

    results.append(site_result)

# 📌 Final DataFrame
results_df = pd.DataFrame(results)

# 📤 Export to CSV
results_df.to_csv("All_Indices_MK_Trend.csv", index=False)

# Optional: Display
results_df.head()

,Sr_no,latitude,longitude,ndvi_trend,ndvi_p_value,ndvi_tau,ndvi_slope,ndvi_cv,ndvi_pettitt_cp,ndvi_pettitt_p,...,bsi_pettitt_h,ndbi_trend,ndbi_p_value,ndbi_tau,ndbi_slope,ndbi_cv,ndbi_pettitt_cp,ndbi_pettitt_p,ndbi_pettitt_u,ndbi_pettitt_h
0,1,27.216662,88.425979,no trend,0.763891,-0.142857,-0.007127,6.826544,5,0.09985,...,True,no trend,0.133128,0.523810,0.021415,-18.901744,4,0.000,12.0,True
1,2,27.216662,88.425979,no trend,0.763891,-0.142857,-0.007127,6.826544,5,0.09945,...,True,no trend,0.133128,0.523810,0.021415,-18.901744,4,0.000,12.0,True
2,3,27.216752,88.425979,no trend,0.229556,-0.428571,-0.005821,2.898196,4,0.00000,...,True,increasing,0.035498,0.714286,0.014138,-7.475474,4,0.000,12.0,True
3,4,27.216662,88.425979,no trend,0.763891,-0.142857,-0.007127,6.826544,5,0.09720,...,True,no trend,0.133128,0.523810,0.021415,-18.901744,4,0.000,12.0,True
4,5,27.213339,88.429932,no trend,0.133128,0.523810,0.019917,6.623541,4,0.28690,...,False,no trend,0.763891,-0.142857,-0.004633,-15.591955,1,0.496,6.0,False


In [ ]:
#NDVI plot
import pandas as pd
import plotly.express as px

# Load your CSV file (replace with correct path)
file_path = '/content/AllIndices_TimeSeries_PerSite.csv'  # Update this path
df = pd.read_csv(file_path)

# Convert 'year' to string so Plotly treats it as a category
df['year'] = df['year'].astype(str)

# Create scatter plots faceted by year (7 plots in 2 rows)
fig = px.scatter(
    df,
    x='Sr_no',             # Point ID or identifier
    y='ndvi',              # NDVI values
    color='Sr_no',         # Color by point ID
    facet_col='year',      # One column per year
    facet_col_wrap=4,      # Wrap after 4 plots => 2 rows (4+3)
    title='NDVI per Point (Faceted by Year)',
    labels={'ndvi': 'NDVI', 'Sr_no': 'Point ID'}
)

fig.update_layout(
    height=600, width=1200,
    showlegend=False
)

fig.show()


In [ ]:
#NDBI plot
import pandas as pd
import plotly.express as px

# Load your CSV file (replace with correct path)
file_path = '/content/AllIndices_TimeSeries_ComparePerSite.csv'  # Update this path
df = pd.read_csv(file_path)

# Convert 'year' to string so Plotly treats it as a category
df['year'] = df['year'].astype(str)

# Create scatter plots faceted by year (7 plots in 2 rows)
fig = px.scatter(
    df,
    x='Sr_no',             # Point ID or identifier
    y='ndvi',              # NDVI values
    color='Sr_no',         # Color by point ID
    facet_col='year',      # One column per year
    facet_col_wrap=4,      # Wrap after 4 plots => 2 rows (4+3)
    title='NDBI per Point (Faceted by Year)',
    labels={'ndvi': 'NDVI', 'Sr_no': 'Point ID'}
)

fig.update_layout(
    height=600, width=1200,
    showlegend=False
)

fig.show()
